In [12]:
#os.environ["TF_CPP_MIN_LOG_LEVEL"] = "0"
from N2_dataprep_time import *
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Flatten, TimeDistributed, Conv1D, MaxPooling1D

In [13]:
epochs = 1
frame_in = 24
frame_out = 2
test_ratio = 0.2

In [14]:
class CNNLSTM(Model):
    def __init__(self):
        super().__init__()

    def setup(self, frame_in, frame_out, filters_=24, kernel_size_=5, pool_size_=4):
        self.model = Sequential()
        self.model.add(TimeDistributed(Conv1D(filters=filters_, kernel_size=kernel_size_, activation='relu'), input_shape=(None, frame_in, 1)))
        self.model.add(TimeDistributed(MaxPooling1D(pool_size=pool_size_)))
        self.model.add(TimeDistributed(Flatten()))
        self.model.add(LSTM(24, activation='relu'))
        self.model.add(Dense(frame_out))
        self.model.compile(optimizer='adam', loss='mse')

In [15]:
str_fn = '-'+str(frame_in)+'-'+str(frame_out)
try:
    train_inp = pk.load(open("temp/trinp"+str_fn, 'rb'))
    train_oup = pk.load(open("temp/troup"+str_fn, 'rb'))
    test_inp = pk.load(open("temp/teinp"+str_fn, 'rb'))
    test_oup = pk.load(open("temp/teoup"+str_fn, 'rb'))
except:
    train_inp, train_oup, test_inp, test_oup = compound(setup(test_ratio, frame_in, frame_out))
    pk.dump(train_inp, open("temp/trinp"+str_fn, 'wb'))
    pk.dump(train_oup, open("temp/troup"+str_fn, 'wb'))
    pk.dump(test_inp, open("temp/teinp"+str_fn, 'wb'))
    pk.dump(test_oup, open("temp/teoup"+str_fn, 'wb'))

In [16]:
train_limit = len(train_inp) // 20
test_limit = len(test_inp) // 20

train_limited_inp = train_inp[:train_limit]
train_limited_oup = train_oup[:train_limit]
test_limited_inp = test_inp[:test_limit]
test_limited_oup = test_oup[:test_limit]

c = CNNLSTM()
c.setup(frame_in, frame_out)

c.setData(train_limited_inp, train_limited_oup)
# c.summarize()

In [17]:
c.train(epochs_=epochs, verbose_=2, test=(train_limited_inp, train_limited_oup))

53/53 - 1s - loss: 0.0016 - val_loss: 1.2205e-04


In [18]:
def testCase(test_inp_, test_oup_, model_, frame_in_, frame_out_, epochs__, label_):
    p = Prediction(test_inp_, test_oup_, model_, frame_in_, frame_out_)
    p.predict()
    p.summary(label=label_, epochs_=epochs__)

testCase(test_limited_inp, test_limited_oup, c, frame_in, frame_out, epochs, 'Global')

62 | ||||||||Global | TD 3.89194 | TC 62 | AD 0.06 | [9.48]%


In [19]:
def truncateSet(sets):
    truncated = []
    for s in sets:
        truncated.append([s[2],s[3]])
    return truncated

In [20]:
# Local prediction
sets = truncateSet(setup(test_ratio, frame_in, frame_out))

In [21]:
for s in sets:
    testCase(s[0][:test_limit], s[1][:test_limit], c, frame_in, frame_out, epochs, str(hash(str(s)))[1:7])

62 | ||||||||667587 | TD 3.89194 | TC 62 | AD 0.06 | [9.48]%
62 | ||||||||463443 | TD 5.42148 | TC 62 | AD 0.08 | [9.56]%
62 | ||||||||224726 | TD 4.46448 | TC 62 | AD 0.07 | [6.36]%
62 | ||||||||463676 | TD 26.77619 | TC 62 | AD 0.43 | [49.81]%
62 | ||||||||129853 | TD 2.34798 | TC 62 | AD 0.03 | [57.63]%
62 | ||||||||779492 | TD 16.02940 | TC 62 | AD 0.25 | [39.66]%
62 | ||||||||215376 | TD 41.04524 | TC 62 | AD 0.66 | [42.87]%
62 | ||||||||409790 | TD 16.81375 | TC 62 | AD 0.27 | [48.37]%
62 | ||||||||867992 | TD 1.66374 | TC 62 | AD 0.02 | [94.51]%
62 | ||||||||141232 | TD 16.19925 | TC 62 | AD 0.26 | [40.03]%
62 | ||||||||722173 | TD 0.21198 | TC 62 | AD 0.0 | [1166.41]%
62 | ||||||||676153 | TD 6.12209 | TC 62 | AD 0.09 | [20.09]%
62 | ||||||||314186 | TD 10.18646 | TC 62 | AD 0.16 | [30.27]%
62 | ||||||||439818 | TD 3.26824 | TC 62 | AD 0.05 | [27.85]%
62 | ||||||||783449 | TD 10.13170 | TC 62 | AD 0.16 | [30.78]%
